# 🤗 Transformers Episodio 4 - Parte 2

Distill Bert


In [2]:
import utils
import importlib
importlib.reload(utils)

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from utils import get_submission, submit, get_tokenizer_and_model,\
                  compute_metrics, load_dfs, tokenize, get_train_args


import transformers
from transformers import Trainer
transformers.logging.set_verbosity_warning()

def read_scores():
    s = !kaggle competitions submissions nlp-getting-started 
    df = pd.DataFrame([(l.split()[0], l.split()[-2]) for l in s[2:]], columns=["Archivo", "Score"]).set_index("Archivo")
    display(df)
    return df

utils.nb_set_width()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
model_id = "episodio_4_pred_2"
pred_file_name = "episodio_4_pred_2.csv"
submit_message = "Episodio 4 Prediccion 2 Distill Bert"

model_name = "distilbert-base-cased"
tokenizer, model = get_tokenizer_and_model(model_name)

df_base, df_test = load_dfs()
df_train, df_val = train_test_split(df_base, test_size=0.066)
ds_train, ds_val, ds_test = tokenize(tokenizer, df_train, df_val, df_test)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier

In [4]:
df_train.shape

(7110, 2)

In [5]:
df_val.shape

(503, 2)

In [1]:
# import torch
#del model
#del trainer
#torch.cuda.empty_cache()

In [6]:
trainer = Trainer(model=model, args=get_train_args(model_id, epochs=4), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

trainer.train()

df_res = get_submission(trainer, ds_test, pred_file_name)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.511600,0.420952,0.831014,0.788030,0.863388,0.724771
2,0.371700,0.470815,0.838966,0.801956,0.858639,0.752294
3,0.258100,0.594695,0.834990,0.800000,0.842640,0.761468
4,0.156500,0.834143,0.811133,0.784580,0.775785,0.793578


In [11]:
submit(pred_file_name, "Distil Bert 4 epochs - Overfitted")

stdout=
Successfully submitted to Natural Language Processing with Disaster Tweets
stderr=
100%|██████████| 22.2k/22.2k [00:00<00:00, 122kB/s]



In [13]:
# Leaderbord - 744
read_scores();

,Score
Archivo,
episodio_4_pred_2.csv,0.81734
episodio_4_pred_1.csv,0.81274
bert_first_submission.csv,0.80049


In [15]:
!ls -l /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/

total 28
drwxr-xr-x 2 dataista dataista 4096 Jun 17 23:50 checkpoint-1000
drwxr-xr-x 2 dataista dataista 4096 Jun 17 23:55 checkpoint-1500
drwxr-xr-x 2 dataista dataista 4096 Jun 17 23:59 checkpoint-2000
drwxr-xr-x 2 dataista dataista 4096 Jun 18 00:03 checkpoint-2500
drwxr-xr-x 2 dataista dataista 4096 Jun 18 00:07 checkpoint-3000
drwxr-xr-x 2 dataista dataista 4096 Jun 18 00:10 checkpoint-3500
drwxr-xr-x 2 dataista dataista 4096 Jun 17 23:46 checkpoint-500


In [14]:
# Un step es un batch
df_train.shape[0]

7110

In [16]:
# Steps por epoch
3556 / 4

889.0

In [8]:
# Steps por epoch (889) por batch size (8) son la cantidad de samples (OK!)
889.0 * 8

7112.0

In [17]:
# Entonces, 2 epocs es 1778 steps, vamos a cargar ese checkpoint
889.0 * 2

1778.0

In [26]:
import transformers
transformers.logging.set_verbosity_debug()
from transformers import AutoModelForSequenceClassification

In [27]:


cp = "/home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-1500"
model = AutoModelForSequenceClassification.from_pretrained(cp, num_labels=2)

trainer = Trainer(model=model, args=get_train_args("test", epochs=2), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

#trainer.train()

#df_res = get_submission(trainer, ds_test, pred_file_name)

trainer.evaluate()

loading configuration file /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-1500/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.7.0.dev0",
  "vocab_size": 28996
}

loading weights file /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-1500/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of Disti

{'eval_loss': 0.4690554141998291,
 'eval_accuracy': 0.827037773359841,
 'eval_f1': 0.7913669064748201,
 'eval_precision': 0.8291457286432161,
 'eval_recall': 0.7568807339449541,
 'eval_runtime': 7.7057,
 'eval_samples_per_second': 65.277,
 'eval_steps_per_second': 8.176}

In [28]:
cp = "/home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-2000"
model = AutoModelForSequenceClassification.from_pretrained(cp, num_labels=2)

trainer = Trainer(model=model, args=get_train_args("test", epochs=2), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

#trainer.train()

#df_res = get_submission(trainer, ds_test, pred_file_name)

trainer.evaluate()

loading configuration file /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-2000/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.7.0.dev0",
  "vocab_size": 28996
}

loading weights file /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-2000/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of Disti

{'eval_loss': 0.6379884481430054,
 'eval_accuracy': 0.8250497017892644,
 'eval_f1': 0.8026905829596411,
 'eval_precision': 0.7850877192982456,
 'eval_recall': 0.8211009174311926,
 'eval_runtime': 7.7217,
 'eval_samples_per_second': 65.141,
 'eval_steps_per_second': 8.159}

In [34]:
cp = "/home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-2000"
model = AutoModelForSequenceClassification.from_pretrained(cp, num_labels=2)

trainer = Trainer(model=model, args=get_train_args("test", epochs=2), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

#trainer.train()

#df_res = get_submission(trainer, ds_test, pred_file_name)

trainer.evaluate()

loading configuration file /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-2000/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.7.0.dev0",
  "vocab_size": 28996
}

loading weights file /home/dataista/git/twitch-streams/data/models/episodio_4_pred_2/checkpoint-2000/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of Disti

{'eval_loss': 0.6379884481430054,
 'eval_accuracy': 0.8250497017892644,
 'eval_f1': 0.8026905829596411,
 'eval_precision': 0.7850877192982456,
 'eval_recall': 0.8211009174311926,
 'eval_runtime': 7.9908,
 'eval_samples_per_second': 62.947,
 'eval_steps_per_second': 7.884}

In [35]:
get_submission(trainer, ds_test, file_name="episodio_4_pred_2_checkpoint_2000.csv")

***** Running Prediction *****
  Num examples = 3263
  Batch size = 8


,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [36]:
submit("episodio_4_pred_2_checkpoint_2000.csv", "Checkpoint 2000 de DistilBert")

stdout=
Successfully submitted to Natural Language Processing with Disaster Tweets
stderr=
100%|██████████| 22.2k/22.2k [00:00<00:00, 115kB/s]



In [38]:
read_scores();

,Score
Archivo,
episodio_4_pred_2_checkpoint_2000.csv,0.80968
episodio_4_pred_2.csv,0.81734
episodio_4_pred_1.csv,0.81274
bert_first_submission.csv,0.80049
